# Gather

### Introduction

This data comes from [The Bureau of Transportation Statistics](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=) and tracks destinations, distance, and delay information of flights across U.S. For this project, I chose the years 2009 through 2017 resulting in 54,383,096 rows of data. To see a more detailed description of the date, click [here](https://www.transtats.bts.gov/Fields.asp?table_id=236).

### Notes

Throughout this notebook, you'll notice that most operations are single cell, this was because my machine kept running out of usable memory. The dataset was really large and required as much memory as possible for each operation. You will see multiple cells of code that could possibly be more code efficient, but were lengthened to reduce memory errors. You will also notice that after certain blocks of code, I outputted the dataframe to a csv. Some blocks of code took more than an hour to run. So to save my progress and make sure that I would not waste time re-running previous code, I would output my progress to a csv and then read it back in. I did not include these datasets in the repository, but have kept the code in the notebook.

The BTS website did not allow users to download full years of data. I downloaded data from the BTS website month by month from 2009 to 2017 and then used `pandas` to merge the data together. If you are interested in downloading the merged data as well as the cleaned data [click here](https://drive.google.com/open?id=1L1jx_CgARXZ1fpgetbvbON9a5AAr19XO).

Finally, after the cleaning process I ended up with 53,068,460 rows resulting in keeping 97% of the data. I was also able to bring the memory usage of the dataset from 11.8 GB to 5.7 GB.

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
file_list = os.listdir('month_data')

In [3]:
combined_csv = pd.concat([pd.read_csv('month_data/' + f) for f in file_list])

C:\Users\kster\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
# change all columns to lowercase
combined_csv.columns = map(str.lower, combined_csv.columns)

In [5]:
# reset index
combined_csv.index = range(len(combined_csv.index))

In [6]:
# drop unnamed column
combined_csv = combined_csv.drop('unnamed: 29', axis = 1)

In [11]:
# output to csv to save memory
combined_csv.to_csv('combined_df.csv', index = False)

# Assess

In [3]:
df = pd.read_csv('combined_df.csv')

C:\Users\kster\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,year,month,day_of_month,day_of_week,unique_carrier,tail_num,fl_num,origin,dest,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2009,1,13,2,DL,N688DL,1237,ATL,RSW,1451.0,1447.0,-4.0,23.0,5.0,1636.0,1656.0,20.0,0.0,NaN,0.0,105.0,129.0,101.0,515.0,0.0,0.0,20.0,0.0,0.0
1,2009,1,13,2,DL,N3752,1239,SLC,PDX,2020.0,2027.0,7.0,17.0,5.0,2127.0,2127.0,0.0,0.0,NaN,0.0,127.0,120.0,98.0,630.0,NaN,NaN,NaN,NaN,NaN
2,2009,1,13,2,DL,N913DN,1240,SLC,ORD,1055.0,1052.0,-3.0,25.0,16.0,1523.0,1507.0,-16.0,0.0,NaN,0.0,208.0,195.0,154.0,1249.0,NaN,NaN,NaN,NaN,NaN
3,2009,1,13,2,DL,N915DN,1241,DTW,SLC,1640.0,1638.0,-2.0,13.0,4.0,1843.0,1833.0,-10.0,0.0,NaN,0.0,243.0,235.0,218.0,1481.0,NaN,NaN,NaN,NaN,NaN
4,2009,1,13,2,DL,N915DN,1242,BOI,SLC,755.0,749.0,-6.0,18.0,6.0,907.0,903.0,-4.0,0.0,NaN,0.0,72.0,74.0,50.0,291.0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54383096 entries, 0 to 54383095
Data columns (total 29 columns):
year                   int64
month                  int64
day_of_month           int64
day_of_week            int64
unique_carrier         object
tail_num               object
fl_num                 int64
origin                 object
dest                   object
crs_dep_time           float64
dep_time               float64
dep_delay              float64
taxi_out               float64
taxi_in                float64
crs_arr_time           float64
arr_time               float64
arr_delay              float64
cancelled              float64
cancellation_code      object
diverted               float64
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
distance               float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
d

* combine `year`, `month`, `day-of-month`, and `day_of_week` into one column in DateTime format
* turn `fl_num` into string
* `crs_arr_time`, and `arrtime` are in hhmm format, change it to standard time
* `cancelled` to bool, 1 = True, 0 = False
* `cancellation_code` has values of NaN, A, B, C, and D. Each needs to be changed.
* `diverted` to bool, 1 = True, 0 = False

# Clean

### Define

* Drop `day_of_week`, `tail_num` columns, they are not needed
* To change `crs_dep_time`, `dep_time`, `crs_arr_time`, and `arr_time` to a standard time format, we need to turn them into integers, then into a different format.
    * turn into int
    * then turn string
    * use `year`, `month`, `day_of_month` to do below steps
    * use `datetime.datetime.strptime('orginal_format', '%H:%M').strftime('%I:%M %p')`
    * then `df['datetime'] = df['year'].map(str) + ' ' + df['month'].map(str) + ' ' + df['day_of_month'].map(str) + ' ' + df['crs_arr_time'].map(str)`
    * if there are errors, drop the row
    * drop `year`, `month`, `day_of_month`, `crs_dep_time`, `dep_time`, `crs_arr_time`, and `arr_time`, these columns will be combined into 4 datetime columns
* drop all flights that were cancelled
    * where `cancelled` = 1.0
    * drop row
* drop all flights that were diverted
    * where `diverted` = 1.0
    * drop row
* combine delay columns into one column
    * combine `carrier_delay`, `nas_delay`, `weather_delay`, `security_delay`, and `late_aircraft_delay`
        * create a column that holds the value 0.0
        * loop through the above columns, if column, row, has value, add to created column
        * if no value, continue loop
        * create similar loop that creates a "label" column
    * drop above columns
* `diverted` to bool, 1 = True, 0 = False
    * use `df[column] = df[column].replace(replacements_dict)`

### Code

In [6]:
# drop day_of_week
df.drop('day_of_week', axis = 1, inplace = True)

In [7]:
# drop tail_num
df.drop('tail_num', axis = 1, inplace = True)

In [8]:
# drop flights that were cancelled
df = df.loc[df.cancelled != 1.0]

In [9]:
# drop cancelled row
df.drop('cancelled', axis = 1, inplace = True)

In [10]:
# drop flights that were diverted
df = df.loc[df.diverted != 1.0]

In [11]:
# drop diverted row
df.drop('diverted', axis = 1, inplace = True)

In [12]:
# drop outliers with missing data
df.dropna(subset=['crs_dep_time', 'taxi_in', 'crs_arr_time', 'arr_time', 
                       'arr_delay', 'crs_elapsed_time', 'actual_elapsed_time', 
                       'air_time'], inplace = True)

In [13]:
# turn crs_arr_time, arr_time, crs_dep_time, dep_time to int
df['crs_arr_time'] = df['crs_arr_time'].astype(int)

In [14]:
df['arr_time'] = df['arr_time'].astype(int)

In [15]:
df['crs_dep_time'] = df['crs_dep_time'].astype(int)

In [16]:
df['dep_time'] = df['dep_time'].astype(int)

In [17]:
# now turn them into strings
df['crs_arr_time'] = df['crs_arr_time'].astype(str)

In [18]:
df['arr_time'] = df['arr_time'].astype(str)

In [15]:
df['crs_dep_time'] = df['crs_dep_time'].astype(str)

In [4]:
df['dep_time'] = df['dep_time'].astype(str)

In [3]:
# turn columns into standard time format
def std_time(string):
    try:
        return datetime.datetime.strptime(string, '%H%M').strftime('%I:%M%p')
    except:
        dep_errors.append(string)

In [23]:
crs_arr_errors = []
df['crs_arr_time'] = df['crs_arr_time'].apply(lambda x: std_time(x))
print('there were %s errors' % len(crs_arr_errors))

there were 105688 errors


In [27]:
# drop rows that had errors in time conversion
df = df[~df['crs_arr_time'].isin(crs_arr_errors)]

In [29]:
arr_errors = []
df['arr_time'] = df['arr_time'].apply(lambda x: std_time(x))
print('there were %s errors' % len(arr_errors))

there were 0 errors


In [34]:
# to csv to save memory
df.to_csv('clean_df_1.csv', index = False)

In [7]:
df = pd.read_csv('clean_df_1.csv')

In [21]:
crs_dep_errors = []
df['crs_dep_time'] = df['crs_dep_time'].apply(lambda x: std_time(x))
print('there were %s errors' % len(crs_dep_errors))

there were 4063 errors


In [23]:
df = df[~df['crs_dep_time'].isin(crs_dep_errors)]

In [28]:
#to csv to save memory
df.to_csv('clean_df_2.csv', index = False)

In [2]:
df = pd.read_csv('clean_df_2.csv')

In [6]:
dep_errors = []
df['dep_time'] = df['dep_time'].apply(lambda x: std_time(x))
print('there were %s errors' % len(dep_errors))

there were 35232 errors


In [7]:
df = df[~df['dep_time'].isin(dep_errors)]

In [9]:
# combine values from year, month, day_of_month, and time column
df['datetime_crs_arr_time'] = df['year'].map(str) + ' ' + df['month'].map(str) + ' ' + df['day_of_month'].map(str) + ' ' + df['crs_arr_time']

In [10]:
df['datetime_arr_time'] = df['year'].map(str) + ' ' + df['month'].map(str) + ' ' + df['day_of_month'].map(str) + ' ' + df['arr_time']

In [11]:
df['datetime_crs_dep_time'] = df['year'].map(str) + ' ' + df['month'].map(str) + ' ' + df['day_of_month'].map(str) + ' ' + df['crs_dep_time']

In [12]:
df['datetime_dep_time'] = df['year'].map(str) + ' ' + df['month'].map(str) + ' ' + df['day_of_month'].map(str) + ' ' + df['dep_time']

In [14]:
df.to_csv('clean_df_3.csv', index = False)

In [2]:
df = pd.read_csv('clean_df_3.csv')

In [4]:
# turn columns into datetime
df['datetime_crs_arr_time'] = pd.to_datetime(df['datetime_crs_arr_time'])

In [7]:
df['datetime_arr_time'] = pd.to_datetime(df['datetime_arr_time'])

In [9]:
df['datetime_crs_dep_time'] = pd.to_datetime(df['datetime_crs_dep_time'])

In [11]:
df['datetime_dep_time'] = pd.to_datetime(df['datetime_dep_time'])

In [ ]:
# to csv to save memory
df.to_csv('clean_df_4.csv', index = False)

In [3]:
df = pd.read_csv('clean_df_4.csv')

In [5]:
# drop year, month, day_of_month, crs_dep_time, dep_time, crs_arr_time, cancellation_code, and arr_delay
df.drop('year', axis = 1, inplace = True)

In [6]:
df.drop('month', axis = 1, inplace = True)

In [7]:
df.drop('day_of_month', axis = 1, inplace = True)

In [8]:
df.drop('crs_dep_time', axis = 1, inplace = True)

In [9]:
df.drop('dep_time', axis = 1, inplace = True)

In [10]:
df.drop('crs_arr_time', axis = 1, inplace = True)

In [11]:
df.drop('arr_time', axis = 1, inplace = True)

In [12]:
df.drop('cancellation_code', axis = 1, inplace = True)

In [13]:
# fill nan values in the delay columns with zero's
df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(0)

In [20]:
# combine delay columns into one label column
cols = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']

# make a new no delay column
df['delay_label'] = 'no_delay'

# loop through columns, if all are 0, move on
# if not, set cell to delay label
for i in df.index:
    for col in cols:
        if df.at[i, col] <= 0.0:
            continue
        else:
            df.at[i, 'delay_label'] = col

In [21]:
df['delay_amount'] = 0.0

# loop through columns, if all are 0, move on
# if not, set cell to the original column/cell value
for i in df.index:
    for col in cols:
        if df.at[i, col] <= 0.0:
            continue
        else:
            df.at[i, 'delay_amount'] = df[col].iloc[i]

In [23]:
df.to_csv('clean_df_5.csv', index = False)

In [2]:
df = pd.read_csv('clean_df_5.csv')

In [6]:
# drop carrier_delay, weather_delay, nas_delay, security_delay, and late_aircraft_delay
df.drop('carrier_delay', axis = 1, inplace = True)

In [7]:
df.drop('weather_delay', axis = 1, inplace = True)

In [8]:
df.drop('nas_delay', axis = 1, inplace = True)

In [9]:
df.drop('security_delay', axis = 1, inplace = True)

In [10]:
df.drop('late_aircraft_delay', axis = 1, inplace = True)

In [15]:
# drop rows that have missing data in the time columns
df.dropna(subset=['datetime_crs_arr_time', 'datetime_arr_time', 'datetime_crs_dep_time', 'datetime_dep_time'], inplace = True)

In [18]:
# turn float columns to int
df['dep_delay'] = df['dep_delay'].astype(int)

In [19]:
df['taxi_out'] = df['taxi_out'].astype(int)

In [20]:
df['taxi_in'] = df['taxi_in'].astype(int)

In [21]:
df['arr_delay'] = df['arr_delay'].astype(int)

In [22]:
df['crs_elapsed_time'] = df['crs_elapsed_time'].astype(int)

In [23]:
df['actual_elapsed_time'] = df['actual_elapsed_time'].astype(int)

In [24]:
df['air_time'] = df['air_time'].astype(int)

In [25]:
df['distance'] = df['distance'].astype(int)

In [26]:
df['delay_amount'] = df['delay_amount'].astype(int)

### Test

In [27]:
df.head(10)

,unique_carrier,fl_num,origin,dest,dep_delay,taxi_out,taxi_in,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,datetime_crs_arr_time,datetime_arr_time,datetime_crs_dep_time,datetime_dep_time,delay_label,delay_amount
0,DL,1237,ATL,RSW,-4,23,5,20,105,129,101,515,2009-01-13 16:36:00,2009-01-13 16:56:00,2009-01-13 14:51:00,2009-01-13 14:47:00,nas_delay,20
1,DL,1239,SLC,PDX,7,17,5,0,127,120,98,630,2009-01-13 21:27:00,2009-01-13 21:27:00,2009-01-13 20:20:00,2009-01-13 20:27:00,no_delay,0
2,DL,1240,SLC,ORD,-3,25,16,-16,208,195,154,1249,2009-01-13 15:23:00,2009-01-13 15:07:00,2009-01-13 10:55:00,2009-01-13 10:52:00,no_delay,0
3,DL,1241,DTW,SLC,-2,13,4,-10,243,235,218,1481,2009-01-13 18:43:00,2009-01-13 18:33:00,2009-01-13 16:40:00,2009-01-13 16:38:00,no_delay,0
4,DL,1242,BOI,SLC,-6,18,6,-4,72,74,50,291,2009-01-13 09:07:00,2009-01-13 09:03:00,2009-01-13 07:55:00,2009-01-13 07:49:00,no_delay,0
5,DL,1242,SLC,DTW,-6,27,10,-8,224,222,185,1481,2009-01-13 15:44:00,2009-01-13 15:36:00,2009-01-13 10:00:00,2009-01-13 09:54:00,no_delay,0
6,DL,1243,CVG,DEN,115,24,8,116,185,186,154,1069,2009-01-13 09:50:00,2009-01-13 11:46:00,2009-01-13 08:45:00,2009-01-13 10:40:00,nas_delay,1
7,DL,1248,MCO,CVG,-8,21,5,-16,138,130,104,756,2009-01-13 16:05:00,2009-01-13 15:49:00,2009-01-13 13:47:00,2009-01-13 13:39:00,no_delay,0
8,DL,1249,CVG,MCO,8,13,8,20,129,141,120,756,2009-01-13 16:14:00,2009-01-13 16:34:00,2009-01-13 14:05:00,2009-01-13 14:13:00,late_aircraft_delay,8
9,DL,1250,MCO,CVG,0,14,5,-10,130,120,101,756,2009-01-13 19:10:00,2009-01-13 19:00:00,2009-01-13 17:00:00,2009-01-13 17:00:00,no_delay,0


In [16]:
df.isna().sum()

unique_carrier           0
fl_num                   0
origin                   0
dest                     0
dep_delay                0
taxi_out                 0
taxi_in                  0
arr_delay                0
crs_elapsed_time         0
actual_elapsed_time      0
air_time                 0
distance                 0
datetime_crs_arr_time    0
datetime_arr_time        0
datetime_crs_dep_time    0
datetime_dep_time        0
delay_label              0
delay_amount             0
dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53068460 entries, 0 to 53396364
Data columns (total 18 columns):
unique_carrier           object
fl_num                   int64
origin                   object
dest                     object
dep_delay                int32
taxi_out                 int32
taxi_in                  int32
arr_delay                int32
crs_elapsed_time         int32
actual_elapsed_time      int32
air_time                 int32
distance                 int32
datetime_crs_arr_time    object
datetime_arr_time        object
datetime_crs_dep_time    object
datetime_dep_time        object
delay_label              object
delay_amount             int32
dtypes: int32(9), int64(1), object(8)
memory usage: 5.7+ GB


In [29]:
# clean dataset to master csv file
df.to_csv('flights_master.csv', index = False)